<a href="https://colab.research.google.com/github/christophecudel/demo_CNN/blob/main/demo_cnn_01_(N_Classes).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Etape 1 : organniser les données pour les rendre
## compatibles avec l'apprentissage du CNN

# Organniser les dossiers comme suit :
# DataBase |- Train |---- Classe_0
#          |        |---- Classe_1
#          |-- Test |---- Classe_0
#                   |---- Classe_1

# A partir des images "Classe_0" et "Classe_1" :
# Environ 80% des images vont dans Train
# et 20% dans Test

# Une fois le dossier correctement constitué :
# - faire un .zip
# - télécharger le .zip avec la ligne ci-dessous

import glob
from google.colab import files

uploaded = files.upload()

In [ ]:
# le fichier est décompressé dans l'envirronnement local
!unzip DataBase_05.zip # >> my_images_base.zip est le nom de votre fichier qui a été téléchargé

In [ ]:
import os

# Lister le contenu de DataBase_05
# Attention au respect des Majuscules minuscules
os.listdir("/content/DataBase_05")


In [ ]:
# Etape 2 : préparer l'apprentissage
# précisant bien les dossiers à prendre en compte :
# chemins vers vos dossiers d'images d'entraînement et de test
# depuis l'environnement google colab
train_data_dir = "DataBase_05/train" # par exemple - à ajuster
test_data_dir = "DataBase_05/test" # par exemple - à ajuster

# Ici, on vérifie que l'accès aux images est OK :
images_dir = "../content/DataBase_05/train/Classe_0/*.bmp"
images_name = glob.glob(images_dir)
print(images_name)
#print("first and last images name : ",images_name[0],",", images_name[-1])

# Si l'instruction ne retourne pas les noms de fichier
# de la 1ère et dernière image : revoir les étapes pér


In [ ]:
# Etape 3 :On affiche les dimensions des images
from PIL import Image

# Ouvrir l'image PNG
image = Image.open(images_name[0])

# Obtenir les dimensions de l'image
NbCols, NbRows = image.size

if image.mode == 'RGB':
    NbCanaux = 3
elif image.mode == 'L':
    NbCanaux = 1
else:
    NbCanaux = 'Inconnu'  # Pour d'autres modes, la gestion peut être ajoutée selon les besoins

# Afficher les informations
print(f'Nombre de colonnes (largeur): {NbCols}, Nombre de lignes (hauteur): {NbRows}, Nombre de canaux: {NbCanaux}')


In [ ]:
# Etape 4 : apprentissage
# Préparation des paramètres pour la
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

import numpy as np

# Créez des générateurs de données distincts pour chaque classe d'entraînement
image_size = (NbRows, NbCols)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',  # Deux classes : classe_0 et classe_1
    shuffle=True
)

# S'il y a plus de classes, il ne faut plus être en "class_mode".
# Fusionnez les générateurs de données en un seul pour l'entraînement en utilisant numpy
#train_generator = np.concatenate([train_generator_classe_0, train_generator_classe_1])

# Créez un générateur de données pour charger et prétraiter les images de test
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',  # Deux classes : classe_0 et classe_1
    shuffle=False  # Vous pouvez désactiver le mélange pour garder l'ordre
)

# Créez un modèle CNN
model = keras.Sequential([
    layers.Input(shape=(NbRows, NbCols, 3)),  # Déclaration de l'entrée ici
    #layers.Conv2D(32, (3, 3), activation='relu', input_shape=(NbRows, NbCols, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Deux classes : classe_0 et classe_1 (décision binaire)
])

# Compiler le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
num_epochs = 100

history = model.fit(train_generator, epochs=num_epochs, validation_data=test_generator)

# Évaluer le modèle sur l'ensemble de test
test_loss, test_acc = model.evaluate(test_generator)
print(f'Accuracy on test set: {test_acc}')

# Sauvegarder le modèle si nécessaire
# ces lignes demandent une durée d'excécution non négligeables
# elles sont nécessaires si on excecute le "model" de CNN dans un autre code
# il faut alors re-charger le "model" de CNN
# chaine = f"content/Classif_keras_{num_epochs:02d}_epochs"
# model.save(chaine)

# Afficher les courbes de loss et d'accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.show()

In [ ]:
# Etape facultative
# Uniquement si le "model" est excuté dans un autre notebook jupyter
# ou bien dans un autre contexte
# étape très longue

import zipfile
import os

# Sauvegarder le modèle (CNN) si nécessaire
dossier = f"/models/Classif_keras_{num_epochs:02d}_epochs"
print("Dossier du modèle à télécharger :" + dossier)
model.save(dossier)

# Créer un fichier ZIP
mon_dossier_zip = dossier + ".zip"
# Créer un fichier ZIP
with zipfile.ZipFile(mon_dossier_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, fichiers in os.walk(dossier):
        for fichier in fichiers:
            chemin_complet = os.path.join(root, fichier)
            nom_dans_zip = os.path.relpath(chemin_complet, os.path.dirname(dossier))
            zipf.write(chemin_complet, nom_dans_zip)

        # Ajouter également les dossiers vides
        if not fichiers:
            nom_dossier_vide = os.path.relpath(root, os.path.dirname(dossier)) + '/'
            zipf.write(root, nom_dossier_vide)

# Télécharger le fichier ZIP
files.download(mon_dossier_zip)


In [ ]:
# Etape : préparation pour visualiser le résultat de l'apprentissage
# Les images ne doivent pas être celles utilisées pour l'apprentissage
# donc prendre les images de test en préparant un fichier zip
# le mieux est de mélanger des images de classe_0 et de classe_1 dans un même dossier

import glob
from google.colab import files

uploaded = files.upload()

In [ ]:
# le fichier est décompressé dans l'envirronnement local
!unzip images_test.zip # >> images_test.zip est le nom de votre fichier qui a été téléchargé

In [ ]:
#Etape 5 : le modèle est testé sur des images

from tensorflow.keras.preprocessing import image
import os

# Spécifiez le chemin du dossier contenant les images à prédire
images_directory = "/content/Images_test"

#NbRows, NbCols = 128
Count_classe_0, Count_classe_1 = 0,0

# Créez une liste pour stocker les images et les prédictions
images = []
predictions = []

# Parcourez les fichiers d'images dans le dossier
for filename in os.listdir(images_directory):
    if filename.endswith('.bmp'):
        # Charger l'image pour la prédiction
        image_path = os.path.join(images_directory, filename)
        img = image.load_img(image_path, target_size=(NbRows, NbCols))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0  # Assurez-vous de normaliser l'image comme pendant l'entraînement

       # Faire la prédiction
        pred = model.predict(img_array)
        predictions.append(pred[0][0])

        # Stocker l'imagen sous forme d'un tableau
        images.append(img)

        # Afficher les images dans une seule figure avec une barre de défilement
        num_images = len(images)


In [ ]:
# si ipywidget n'est pas installé :
# outil utilisé pour la visualisation dans l'étape suivante
!pip install ipywidgets

In [ ]:
# Etape 6 : visualisation des résultats sur les images de test
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# Supposons que vous avez une liste d'images et une liste de noms correspondants
#images = [img1, img2, img3, ...]  # Remplacez ceci par vos images

image_names = os.listdir(images_directory)

# Créer un widget Output pour l'affichage des images
output = widgets.Output()

# Créer un slider
slider = widgets.IntSlider(min=0, max=len(images) - 1, step=1, description='Image Index:')

# Fonction pour mettre à jour l'image et le nom
def update_image(change):
    img_index = change['new']  # Utilisation de change['new'] au lieu de change.new
    with output:
        clear_output(wait=True)  # Effacer l'affichage précédent
        plt.figure(figsize=(5,5))  # Ajustez la taille si nécessaire
        plt.imshow(images[img_index])  # Afficher la nouvelle image
        #plt.title(image_names[img_index])  # Afficher le nom de l'image
        plt.axis('off')
        if predictions[img_index]>0.5:
          classe = 'Classe 1'
        else:
          classe = 'Classe 0'
        plt.title(image_names[img_index] + ':'+classe,fontsize=12)
        plt.show()

# Observer les changements du slider
slider.observe(update_image, names='value')

# Afficher le slider et la zone d'affichage
display(slider)
display(output)

# Affichage initial
update_image({'new': slider.value})